# Initial visualization attempt.
We'll import all of the saved data from the previous sim run, then visualize it in an interactive Plotly dashboard.

## TODO: Need to aggregate along sliding windows to turn our non-uniform sampled data into smoother, uniform sampled data.
This will remove the "teleporting" that we see in the travel animations.

First, we'll load up all of our data.

In [1]:
import pandas as pd

mapbox_token = open('./.keys/.mapbox_key').read()

output_data_dir = '../sim/output/'

In [2]:
# Import static object data.
rail_terminal_locations = pd.read_parquet(output_data_dir+'rail_terminal_locations.parquet')
container_yard_locations = pd.read_parquet(output_data_dir+'container_yard_locations.parquet')

# Import dynamic object data.
NUM_PARALLEL_VEHICLES = 1
parallel_vehicles_outputs = {}
for i in range(NUM_PARALLEL_VEHICLES):
    parallel_vehicles_outputs[i] = pd.read_parquet(output_data_dir+f'parallel_vehicle_{i}.parquet')

cordele_outputs = pd.read_parquet('../sim/output/rail_terminal_Cordele Intermodal Services, Inc..parquet')
savannah_outputs = pd.read_parquet('../sim/output/rail_terminal_Savannah Marine Terminal.parquet')

In [3]:
import plotly.graph_objects as go

# fig = go.Figure().set_subplots(1, 2, horizontal_spacing=0.01, specs=[[{"type":"scattermapbox"}, {"type": "scatter"}]])

# Make figure dictionaries
fig_dict = {
    "data": [],
    "layout": {},
    "frames": []
}

# fill in most of layout
fig_dict["layout"]["updatemenus"] = [
    {
        "buttons": [
            {
                "args": [None, {"frame": {"duration": 500, "redraw": True},
                                "fromcurrent": True, "transition": {"duration": 0,
                                                                    "easing": "linear"}}],
                "label": "Play",
                "method": "animate"
            },
            {
                "args": [[None], {"frame": {"duration": 0, "redraw": False},
                                  "mode": "immediate",
                                  "transition": {"duration": 0}}],
                "label": "Pause",
                "method": "animate"
            }
        ],
        "direction": "left",
        "pad": {"r": 10, "t": 10},
        "showactive": False,
        "type": "buttons",
        "x": 0.1,
        "xanchor": "right",
        "y": 0,
        "yanchor": "top"
    }
]

sliders_dict = {
    "active": 0,
    "yanchor": "top",
    "xanchor": "left",
    "currentvalue": {
        "font": {"size": 20},
        "prefix": "Hour:",
        "visible": True,
        "xanchor": "right"
    },
    "transition": {"duration": 300, "easing": "cubic-in-out"},
    "pad": {"b": 10, "t": 10},
    "len": 0.9,
    "x": 0.1,
    "y": 0,
    "steps": []
}

# make initial data
for id, result in parallel_vehicles_outputs.items():
    vehicle_data_dict = {
        "type": "scattermapbox",
        "lat": [result['LATITUDE'][0]],
        "lon": [result['LONGITUDE'][0]],
        "mode": "markers",
        "text": [f'Parallel Vehicle {id}'],
        "marker": go.scattermapbox.Marker({
            "size": 40,
            "symbol": 'circle',
            "color": 'rgb(200, 0, 0)',
            "opacity": 1.0,
        }),
        "showlegend": False,
        "name": f'Parallel Vehicle {id}'
    }
    fig_dict["data"].append(vehicle_data_dict)
    # line_data_dict = {
    #     "type": "scatter",
    #     "x": [cordele_outputs.TIME[0]],
    #     "y": [cordele_outputs.ARRIVAL_QUEUE_SIZE[0]]
    # }
    # fig_dict["data"].append(line_data_dict)

# make sequence of frames
for id, result in parallel_vehicles_outputs.items():
    for index, row in result.iterrows():
        frame = {"data": [], "name": f"{row['TIME']}"}
        vehicle_data_dict = {
            "type": "scattermapbox",
            "lat": [row['LATITUDE']],
            "lon": [row['LONGITUDE']],
            "mode": "markers",
            "text": [f'Parallel Vehicle {id}'],
            "marker": go.scattermapbox.Marker({
                "size": 40,
                "symbol": 'circle',
                "color": 'rgb(200, 0, 0)',
            }),
            "showlegend": False,
            "name": f'Parallel Vehicle {id}'
        }
        frame["data"].append(vehicle_data_dict)

        # line_data_dict = {
        #     "type": "scatter",
        #     "x": [cordele_outputs.TIME[:index]],
        #     "y": [cordele_outputs.ARRIVAL_QUEUE_SIZE[:index]]
        # }
        # frame["data"].append(line_data_dict)

        fig_dict["frames"].append(frame)
        slider_step = {"args": [
            [row['TIME']],
            {"frame": {"duration": 300, "redraw": False},
            "mode": "immediate",
            "transition": {"duration": 300}}
        ],
            "label": f"{row['TIME']:.2f}",
            "method": "animate"}
        sliders_dict["steps"].append(slider_step)

fig_dict["layout"]["sliders"] = [sliders_dict]

fig = go.Figure(fig_dict)

lat_lines = []
lon_lines = []

for _, row in container_yard_locations.iterrows():
    lat_lines.append(rail_terminal_locations['LATITUDE'][1])
    lat_lines.append(row['LATITUDE'])
    lon_lines.append(rail_terminal_locations['LONGITUDE'][1])
    lon_lines.append(row['LONGITUDE'])

fig.add_trace(
    go.Scattermapbox(
        name='',
        mode='lines',
        lat=lat_lines,
        lon=lon_lines,
        showlegend=False,
    )
)

fig.add_trace(
    go.Scattermapbox(
        name='',
        mode='lines',
        lat=[lat for lat in rail_terminal_locations['LATITUDE']],
        lon=[lon for lon in rail_terminal_locations['LONGITUDE']],
        showlegend=False,
    )
)

fig.add_trace(
    go.Scattermapbox(lat=container_yard_locations['LATITUDE'],
                     lon=container_yard_locations['LONGITUDE'],
                     mode='markers',
                     text=container_yard_locations['NAME'],
                     marker=go.scattermapbox.Marker(size=10),
                     showlegend=False,
                     name='Container Yard'
                     ),
)

fig.add_trace(
    go.Scattermapbox(lat=rail_terminal_locations['LATITUDE'],
                    lon=rail_terminal_locations['LONGITUDE'],
                    mode='markers',
                    text=rail_terminal_locations['NAME'],
                    marker=go.scattermapbox.Marker(size=15),
                    showlegend=False,
                    name='Rail Terminal'
                    ),
)

fig.update_mapboxes(
    accesstoken=mapbox_token,
    center=dict(lat=31.97358,
                lon=-82.48189
                ),
    style='outdoors',
    zoom=6.5,
)

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

fig.set_subplots(rows=2,
                 cols=1,
                 horizontal_spacing=0.1)



fig.show()

In [178]:
from dash import Dash, dcc, html

app = Dash(__name__)

app.layout = html.Div([
    dcc.Graph(figure=fig)
])

app.run_server(debug=True, use_reloader=False)